# How to sign your results

In this notebook we collect necessary code snippets to sign messages.

Let us start with an import of the relevant functions

In [9]:
# for key generation etc
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

# to save the private key
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)

# to store the public key
from cryptography.hazmat.primitives.serialization import PublicFormat

# to load the private and public key
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key

The following is the vanilla example from the [official documentation](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed25519/).

In [10]:

private_key = Ed25519PrivateKey.generate()

signature = private_key.sign(b"my authenticated message")

public_key = private_key.public_key()

# Raises InvalidSignature if verification fails

public_key.verify(signature, b"my authenticated message")

Really nice, but now we have to wonder about:

- How to generate and save the key pair ?
- How to store the signature ?
- How to load the keys at the appropriate time ?

# Storing and loading the private key

It would seem to be quite some decision to be made here. We will go the the PEM format, which seems to be fairly broadly used. I am not sure at all what the format changes, but whatever... To be seen...

In [11]:
private_key_file_name = "private_key_test.pem"
private_bytes = private_key.private_bytes(
    encoding=Encoding.PEM, format=PrivateFormat.PKCS8, encryption_algorithm=NoEncryption()
)

with open(private_key_file_name, "wb") as pem_file:
    pem_file.write(private_bytes)

And now it is time to also load the private key.

In [12]:
with open(private_key_file_name, "rb") as pem_file:
    private_bytes_loaded = pem_file.read()

private_key_loaded = load_pem_private_key(private_bytes_loaded, password=None)

## Storing and loading the public key

Let us now also find a way to store and load the public key.

In [13]:
public_key_file_name = "public_key_test.pem"

public_bytes = public_key.public_bytes(
    Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo
)

with open(public_key_file_name, "wb") as pem_file:
    pem_file.write(public_bytes)

and load it again

In [14]:
with open(public_key_file_name, "rb") as pem_file:
    public_bytes_loaded = pem_file.read()

public_key_loaded = load_pem_public_key(public_bytes_loaded)

time to test if the loading worked nicely

In [15]:
signature = private_key_loaded.sign(b"my authenticated message")
print(signature)
public_key_loaded.verify(signature, b"my authenticated message")

b'\x14-\x95\xce\x16>\xee;\xdc\xed\xce\x10\xdc\xdc\xf5\xf6\xce\xad\x9e\x14o\xd1\x02\xc6\\f\x91D\x8b\xbd\xbd\xe3\x03=A\xbb\x9d\xac\rK\xe5sY\xc1\xecG\xb9\xe1;\x9d1\xb63\x14\xd9^\xbf\xa4A-\xbbV\x8b\x0f'


Now  let us move on to the part, where we sign the document. Quite intestingly, we will mainly sign json files. So it might be 
that `jwcrypto` should be used fairly soon anyways. But for now, we will stick to the `cryptography` library.


In [16]:
test_dict = {"name": "daniel", "age": 25}

# let us serialize the dictionary to a json string

import json
test_dict_json = json.dumps(test_dict)
print(test_dict_json)

{"name": "daniel", "age": 25}


And it is time to sign as well as to verify the signature.

In [17]:
signature = private_key_loaded.sign(test_dict_json.encode())

public_key_loaded.verify(signature, test_dict_json.encode())

Now that this works we need to decide on how to attach the signature. Let us already have a look at the result dictionary that we are having in mind at the moment.

## JWS

The JSON Web Signature (JWS) is a compact signature format that is intended for space constrained environments such as HTTP Authorization headers and URI query parameters. It is also useful in many other situations where a JSON data structure needs to be signed or MACed. A nice introduction is given in the [official documentation](https://openid.net/specs/draft-jones-json-web-signature-04.html). However, it is actually a fairly complex format and will take quite some time to be implemented properly around here.

Some interesting and important features are:

- The whole thing has a very well defined header, which is part of the things that should be signed.
- The payload and the header are base64url encoded and concatenated with a dot.
- Both are then signed and the signature is also base64url encoded.

Let us try to see how far we want to go there.



## Simple JSON Signature (SJS)

Let us take the JWS and simply go with the most important features for now. First, is to have a nice header.

### Header

- `alg` : The algorithm used for the signature. We will go with `Ed25519` for now.
- `kid` : The key id. This is the name of the file where the public key is stored. We will go with the name of the file without the extension for now.
- `version`: The version of the signature format. We will go with `0.1` for now.

Other things like the adress etc will have to come later

In [18]:
from pydantic import BaseModel, Field
import base64
class SjsHeader(BaseModel):
    alg: str = Field(default="Ed25519", description="The algorithm used for signing")
    kid: str = Field(description="The key id of the public key used for signing")
    version: str = Field(default = "0.1", description="The version of the signature format")

def header_to_base64url(header: SjsHeader) -> bytes:
    """
    Convert an SJS header to a base64url encoded string.

    Args:
        header : The SJS header to encode

    Returns:
        bytes : The base64url encoded header
    """

    # transform into a json string
    header_json = header.model_dump_json()

    # binary encode the json string
    binary_string = header_json.encode()

    # base64 encode the binary string
    base64_encoded =    base64.b64encode(binary_string)
    return base64_encoded

def payload_to_base64url(payload: dict) -> bytes:
    """
    Convert an arbitrary payload to a base64url encoded string.

    Args:
        payload : The dictionary to encode

    Returns:
        bytes : The base64url encoded header
    """

    # transform into a json string
    payload_string = json.dumps(payload)

    # binary encode the json string
    binary_string = payload_string.encode()

    # base64 encode the binary string
    base64_encoded = base64.b64encode(binary_string)
    return base64_encoded

In [24]:
header = SjsHeader(kid="test_key")

header_base64 = header_to_base64url(header)
print(header_base64)
payload_base64 = payload_to_base64url(test_dict)
print(payload_base64)

full_message = header_base64 + b"." + payload_base64
print(full_message)

signature = private_key.sign(full_message)
signature_base64 = base64.b64encode(signature)
print(signature_base64)

# the full string 

sjs_string = full_message + b"." + signature_base64
print(sjs_string)

b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ2ZXJzaW9uIjoiMC4xIn0='
b'eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0='
b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ2ZXJzaW9uIjoiMC4xIn0=.eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0='
b'j7dPqAwVQWPDoITFat+2VxRxHfn4uaNDF1wPrhZ9vduPj0GBkCvjkn0K6Nxkn2zpgV+FU5TbAamg35X8seQxCA=='
b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ2ZXJzaW9uIjoiMC4xIn0=.eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0=.j7dPqAwVQWPDoITFat+2VxRxHfn4uaNDF1wPrhZ9vduPj0GBkCvjkn0K6Nxkn2zpgV+FU5TbAamg35X8seQxCA=='
